In [62]:
import pandas as pd
from dbHelpers import createEngine
import numpy as np
import uuid

In [63]:
engine = createEngine()

In [101]:
states = pd.read_sql('select * from npd.fips_state', con = engine)
states.set_index('abbreviation', inplace=True)

In [102]:
npis = pd.read_sql('select individual_id, organization_id, n. npi from npd.npi n left join npd.provider p on p.npi=n.npi left join npd.clinical_organization o on o.npi = n.npi', con = engine)
npis.set_index('npi', inplace=True)

In [103]:
df = pd.read_csv('addresses.csv')

In [104]:
df['state'] = [states.loc[s]['id'] for s in df['state']]

In [105]:
df['delivery_line_2'] = [f"Ste {l.split(' Ste ')[1]}" if ' Ste ' in l else np.nan for l in df['delivery_line_1']]

In [106]:
df['delivery_line_1'] = [l.split(' Ste ')[0] if ' Ste ' in l else l for l in df['delivery_line_1']]

In [107]:
dedup_address = df[['id', 'latitude', 'longitude', 'delivery_line_1', 'delivery_line_2', 'city', 'state', 'postalcode', 'record_type']].drop_duplicates()

In [114]:
dedup_address.rename(columns={'postalcode':'zipcode', 'city':'city_name', 'state':'state_code'}).to_sql('address_us', con = engine, if_exists = 'append', index=False, schema='npd')

734

In [115]:
dedup_address.rename(columns={'id':'address_us_id'}, inplace=True)
dedup_address['id'] = [str(uuid.uuid4()) for i in dedup_address.index]
address_ids = dedup_address[['id', 'address_us_id']]

In [116]:
address_ids.to_sql('address', con = engine, if_exists = 'append', index=False, schema='npd')

734

In [117]:
address_ids.set_index('address_us_id', inplace=True)

In [118]:
df['address_id'] = [address_ids.loc[a]['id'] for a in df['id']]

In [119]:
df['organization_id'] = [npis.loc[n]['organization_id'] for n in df['npi']]
df['individual_id'] = [npis.loc[n]['individual_id'] for n in df['npi']]

In [124]:
individual_addresses = df.dropna(subset='individual_id')[['address_id', 'individual_id']]
individual_addresses['address_use_id'] = 2
individual_addresses.drop_duplicates(inplace = True)

In [126]:
individual_addresses.to_sql('individual_to_address', con = engine, if_exists = 'append', index=False, schema='npd')

190

In [127]:
organization_addresses = df.dropna(subset='organization_id')[['address_id', 'organization_id']]
organization_addresses['address_use_id'] = 2
organization_addresses.drop_duplicates(inplace = True)

In [128]:
organization_addresses.to_sql('organization_to_address', con = engine, if_exists = 'append', index=False, schema='npd')

79